# Proxy Variable - Violence and Recidivism Comparison

The code bellow compares correlation and network output in an attempt to identitfy Proxy Varaibles.

In [110]:
from conx import Network
import random

def getData(file):
    #open file
    tf = open(file, "r")
    content = tf.readlines()
    row_count = sum(1 for row in content)
    print(row_count)
    #Parse file
    input = []
    content = [x.strip() for x in content[1:]]
    for y in content:
        row = [col for col in y.split(",")]
        #print(row)
        input.append(row)
    tf.close()
    return input

cols = getData("compas-scores-two-years.csv")


7215


In [111]:
#In column 23 there was an issue with splitting the value stored at ","
#Resulting in a list containing 54 columns instead of 53
for c in cols:
    if len(c) == 54:
        c.remove(c[23])

In [112]:
def parseInputs(inputs):
    new = []
    row = []
    countWhat = 0
    for r in inputs:
        if r[9] == "Caucasian" or r[9] == "African-American":
            if r[5] == "Male":
                row.append(1)
            elif r[5] == "Female":
                row.append(0)
            else:
                print("error parsing col 5, Error occurs on row " + str(countWhat))
            #Age: Greater than 45, 25 - 45, and Less than 25
            if r[8] == "Greater than 45":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[8] == "25 - 45":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[8] == "Less than 25":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 8, Error occurs on row " + str(countWhat))
            #Risk of violence Score: Low, Medium, and High
            if r[40] == "Low":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[40] == "Medium":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[40] == "High":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 44, Error occurs on row " + str(countWhat))
            #Risk of recidi Score: Low, Medium, and High
            if r[44] == "Low":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[44] == "Medium":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[44] == "High":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 44, Error occurs on row " + str(countWhat))
            #If recidivism 
            if r[52] == '0':
                row.append(0)
            elif r[52] == '1':
                row.append(1)
            else:
                print("error parsing col 52, Error occurs on row " + str(countWhat))
            #parse error in row 4754, disposed of here
            if r[40] == "":
                r = []
            if row != [] and len(row) == 11:
                new.append(row)
            row = []
        countWhat += 1
    return new

tempinputs = parseInputs(cols)  

error parsing col 44, Error occurs on row 4754


In [113]:
#Ensure inputs are correct length
#Input how many rows to be processed
inputs = []
lengthInput = 4000
for r in range(lengthInput):
    inputs.append(tempinputs[r])  
    
inputs = [[x[0:10], x[10:]] for x in inputs]

In [114]:
net = Network(10, 60, 1)

#desired output/target
def des(inputs):
    re = inputs[11]
    return [int(re)]

In [115]:
inputs[0]

[[1, 0, 1, 0, 1, 0, 0, 1, 0, 0], [1]]

In [116]:
net.set_inputs(inputs)

In [117]:
net.reset()
#net.train(max_training_epochs=1000, momentum=1, epsilon=1)
net.train(max_training_epochs=1500, report_rate=250, tolerance = .4)

#stop_percentage: default 1.0, # used in train
#tolerance: default 0.1, # used in train
#report_rate: default 500, # used in train
#epsilon: default 0.1, # learning rate, used in train
#momentum: default 0.9, # used in train
#shuffle=True

--------------------------------------------------
Training for max trails: 1500 ...
Epoch: 0 TSS error: 1003.03264802 %correct: 0.26425
Epoch: 250 TSS error: 861.968743526 %correct: 0.50725
Epoch: 500 TSS error: 861.170644959 %correct: 0.50225
Epoch: 750 TSS error: 861.97163362 %correct: 0.49725
Epoch: 1000 TSS error: 862.130861975 %correct: 0.50125
Epoch: 1250 TSS error: 859.870891196 %correct: 0.50475
Epoch: 1500 TSS error: 860.712902815 %correct: 0.50775
--------------------------------------------------
Epoch: 1500 TSS error: 860.712902815 %correct: 0.50775


In [118]:
net.test()

--------------------------------------------------
Test:
******************************
Input : [0, 0, 1, 0, 1, 0, 0, 1, 0, 0]
Output: [ 0.25294127] 
Target: [0] Correct
******************************
Input : [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]
Output: [ 0.22889908] 
Target: [0] Correct
******************************
Input : [0, 0, 0, 1, 0, 0, 1, 0, 0, 1]
Output: [ 0.64355728] 
Target: [1] Correct
******************************
Input : [1, 1, 0, 0, 1, 0, 0, 1, 0, 0]
Output: [ 0.24731493] 
Target: [0] Correct
******************************
Input : [1, 0, 1, 0, 1, 0, 0, 0, 1, 0]
Output: [ 0.51948424] 
Target: [1] Incorrect
******************************
Input : [1, 1, 0, 0, 1, 0, 0, 1, 0, 0]
Output: [ 0.24731493] 
Target: [0] Correct
******************************
Input : [0, 0, 1, 0, 1, 0, 0, 1, 0, 0]
Output: [ 0.25294127] 
Target: [0] Correct
******************************
Input : [1, 0, 0, 1, 0, 1, 0, 0, 1, 0]
Output: [ 0.55454819] 
Target: [1] Incorrect
******************************
Inpu

In [120]:
#try network on test data
results = []
forit = len(tempinputs) - 4000
for h in range(forit):
    val = tempinputs[h + 4000]
    val = val[0:10]
    results.append(net.propagate(val))

NETWORK 2

In [122]:
def parseInputs(inputs):
    new = []
    row = []
    countWhat = 0
    for r in inputs:
        if r[9] == "Caucasian" or r[9] == "African-American":
            #Age: Greater than 45, 25 - 45, and Less than 25
            #1, 2, and 3
            #race = Caucasian, African-American
            if r[9] == "Caucasian":
                row.append(0)
            elif r[9] == "African-American":
                row.append(1)
            else:
                print("error parsing col 9, Error occurs on row " + str(countWhat))
            if r[8] == "Greater than 45":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[8] == "25 - 45":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[8] == "Less than 25":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 8, Error occurs on row " + str(countWhat))
            #Risk of violence Score: Low, Medium, and High
            if r[40] == "Low":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[40] == "Medium":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[40] == "High":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 44, Error occurs on row " + str(countWhat))
            #Risk of recidi Score: Low, Medium, and High
            if r[44] == "Low":
                row.append(1)
                row.append(0)
                row.append(0)
            elif r[44] == "Medium":
                row.append(0)
                row.append(1)
                row.append(0)
            elif r[44] == "High":
                row.append(0)
                row.append(0)
                row.append(1)
            else:
                print("error parsing col 44, Error occurs on row " + str(countWhat))
            #If recidivism 
            if r[52] == '0':
                row.append(0)
            elif r[52] == '1':
                row.append(1)
            else:
                print("error parsing col 52, Error occurs on row " + str(countWhat))
            
            #parse error in row 4754, disposed of here
            if r[40] == "":
                r = []

            if row != [] and len(row) == 11:
                new.append(row)
            row = []
        countWhat += 1
    return new

tempinputs = parseInputs(cols)  

error parsing col 44, Error occurs on row 4754


In [123]:
print(tempinputs[0])

[1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1]


In [124]:
print(cols[0])

['1', 'miguel hernandez', 'miguel', 'hernandez', '2013-08-14', 'Male', '1947-04-18', '69', 'Greater than 45', 'Other', '0', '1', '0', '0', '0', '-1', '2013-08-13 06:03:42', '2013-08-14 05:41:20', '13011352CF10A', '2013-08-13', '', '1', 'F', 'Aggravated Assault w/Firearm', '0', '', '', '', '', '', '', '', '', '0', '', '', '', '', 'Risk of Recidivism', '1', 'Low', '2013-08-14', 'Risk of Violence', '1', 'Low', '2013-08-14', '2014-07-07', '2014-07-14', '0', '0', '327', '0', '0']


In [125]:
#Reset inputs
inputs = []
print(tempinputs[0])
for r in range(lengthInput):
    inputs.append(tempinputs[r])  
    
inputs = [[x[0:10], x[10:]] for x in inputs]

[1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1]


In [126]:
inputs[0]

[[1, 0, 1, 0, 1, 0, 0, 1, 0, 0], [1]]

In [127]:
net2 = Network(10, 60, 1)

#desired output/target
def des(inputs):
    re = inputs[11]
    return [int(re)]

In [128]:
net2.set_inputs(inputs)

In [129]:
net2.reset()
#net.train(max_training_epochs=1000, momentum=1, epsilon=1)
net2.train(max_training_epochs=1500, report_rate=250, tolerance = .4)

--------------------------------------------------
Training for max trails: 1500 ...
Epoch: 0 TSS error: 1006.49811844 %correct: 0.45125
Epoch: 250 TSS error: 874.010174962 %correct: 0.50975
Epoch: 500 TSS error: 875.414060282 %correct: 0.50425
Epoch: 750 TSS error: 871.507221081 %correct: 0.512
Epoch: 1000 TSS error: 874.633372521 %correct: 0.5075
Epoch: 1250 TSS error: 873.464028424 %correct: 0.508
Epoch: 1500 TSS error: 868.666521405 %correct: 0.5185
--------------------------------------------------
Epoch: 1500 TSS error: 868.666521405 %correct: 0.5185


In [130]:
net2.test()

--------------------------------------------------
Test:
******************************
Input : [0, 0, 0, 1, 1, 0, 0, 1, 0, 0]
Output: [ 0.24879281] 
Target: [1] Incorrect
******************************
Input : [1, 0, 1, 0, 0, 1, 0, 1, 0, 0]
Output: [ 0.5189312] 
Target: [1] Incorrect
******************************
Input : [0, 0, 1, 0, 0, 0, 1, 0, 0, 1]
Output: [ 0.71725042] 
Target: [1] Correct
******************************
Input : [1, 0, 1, 0, 0, 0, 1, 0, 1, 0]
Output: [ 0.73118646] 
Target: [0] Incorrect
******************************
Input : [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]
Output: [ 0.21981541] 
Target: [0] Correct
******************************
Input : [1, 0, 0, 1, 0, 0, 1, 0, 0, 1]
Output: [ 0.81968892] 
Target: [1] Correct
******************************
Input : [1, 0, 0, 1, 0, 1, 0, 0, 1, 0]
Output: [ 0.49770195] 
Target: [0] Incorrect
******************************
Input : [0, 1, 0, 0, 1, 0, 0, 1, 0, 0]
Output: [ 0.21981541] 
Target: [1] Incorrect
******************************

Look at data range = len(tempinputs) - 4000

In [131]:
results2 = []
forIt = (len(tempinputs) - 4000)
for p in range(forIt):
    val = tempinputs[p + 4000]
    val = val[0:10]
    results2.append(net2.propagate(val))
    

In [132]:
print("Proxy Precentage: ")  

Proxy Precentage: 


In [133]:
countSame = 0
for s in range(len(results2)):
    if (results[s] > .5 and results2[s] > .5) or (results[s] < .5 and results2[s] < .5):
        countSame += 1
        
print(countSame)
print("Proxy Precentage: ")      
print(float(countSame)/float(len(results)))

1901
Proxy Precentage: 
0.8845974872033504


In [134]:
def getProxy(results, results2):
    countSame = 0
    for s in range(len(results2)):
        bothTrue = results[s] > .5 and results2[s] > .5
        bothFalse = results[s] < .5 and results2[s] < .5
        if bothTrue or bothFalse:
            countSame += 1     
    return (float(countSame)/float(len(results)))

print(getProxy(results, results2))

0.8845974872033504


Check against actual results for true/false?